In [7]:
import os
import glob
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import make_grid, save_image
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, BatchNormalization, ReLU, Conv2DTranspose, Dropout, Concatenate
from tensorflow.keras.models import Model
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
# Install required packages if not already available
!pip install gradio -q

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, ReLU, BatchNormalization, Activation, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import gradio as gr
import kagglehub

In [8]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [14]:
# ----------- Download Dataset using kagglehub -----------
print("Downloading dataset from KaggleHub...")
path = kagglehub.dataset_download("almightyj/person-face-sketches")
print("Dataset path:", path)

Dataset path: /kaggle/input/person-face-sketches


In [15]:
# # Paths
# train_img_dir = r"/kaggle/input/person-face-sketches/train"
# test_img_dir = r"/kaggle/input/person-face-sketches/test"
# val_img_dir = r"/kaggle/input/person-face-sketches/val"

# def list_images(path):
#     return sorted([
#         os.path.join(path, fname)
#         for fname in os.listdir(path)
#         if fname.lower().endswith(('.jpg', '.png'))
#     ])

# def prepare_dataset(photo_path, sketch_path, count=1000):
#     photos = list_images(photo_path)[:count]
#     sketches = list_images(sketch_path)[:count]
#     return photos, sketches

# train_photos, train_sketches = prepare_dataset(
#     os.path.join(train_img_dir, 'photos'),
#     os.path.join(train_img_dir, 'sketches')
# )

# IMG_DIM = (256, 256)

# def load_image(img_path, gray=False):
#     img = load_img(img_path, target_size=IMG_DIM, color_mode='grayscale' if gray else 'rgb')
#     return img_to_array(img) / 127.5 - 1.0

# # Load datasets
# X_train = np.array([load_image(p, gray=True) for p in train_sketches])
# Y_train = np.array([load_image(p) for p in train_photos])

# print(f"Sketches: {X_train.shape}, Photos: {Y_train.shape}")

# Directories
train_dir = "/kaggle/input/person-face-sketches/train"
photo_path = os.path.join(train_dir, "photos")
sketch_path = os.path.join(train_dir, "sketches")

# Load image helper
def load_image(img_path, grayscale=False):
    img = load_img(img_path, target_size=(256, 256), color_mode='grayscale' if grayscale else 'rgb')
    img = img_to_array(img) / 127.5 - 1.0
    return img

def get_data(photo_dir, sketch_dir, count=500):
    photo_files = sorted(os.listdir(photo_dir))[:count]
    sketch_files = sorted(os.listdir(sketch_dir))[:count]
    photos = np.array([load_image(os.path.join(photo_dir, f)) for f in photo_files])
    sketches = np.array([load_image(os.path.join(sketch_dir, f), grayscale=True) for f in sketch_files])
    sketches = np.repeat(sketches, 3, axis=-1)  # make 3 channels
    return photos, sketches

photos, sketches = get_data(photo_path, sketch_path)

print("Photos:", photos.shape, "Sketches:", sketches.shape)

Photos: (500, 256, 256, 3) Sketches: (500, 256, 256, 3)


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np

# ------------------------------
# Generator and Discriminator
# ------------------------------

def build_generator():
    inputs = layers.Input(shape=(256, 256, 3))
    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    x = layers.ReLU()(x)
    x = layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same')(x)
    outputs = layers.Activation('tanh')(x)
    return models.Model(inputs, outputs, name="Generator")

def build_discriminator():
    inputs = layers.Input(shape=(256, 256, 3))
    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Conv2D(1, kernel_size=4, strides=2, padding='same')(x)
    outputs = layers.Activation('sigmoid')(x)
    return models.Model(inputs, outputs, name="Discriminator")

# ------------------------------
# Instantiate models
# ------------------------------

G = build_generator()  # Sketch → Photo
F = build_generator()  # Photo → Sketch
D_X = build_discriminator()  # Real Sketch Discriminator
D_Y = build_discriminator()  # Real Photo Discriminator

# ------------------------------
# Optimizers - separate instances!
# ------------------------------

g_optimizer = optimizers.Adam(2e-4, beta_1=0.5)
f_optimizer = optimizers.Adam(2e-4, beta_1=0.5)
dx_optimizer = optimizers.Adam(2e-4, beta_1=0.5)
dy_optimizer = optimizers.Adam(2e-4, beta_1=0.5)

# ------------------------------
# Loss functions
# ------------------------------

bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
mae = tf.keras.losses.MeanAbsoluteError()

# ------------------------------
# Training Step Function
# ------------------------------

@tf.function
def train_step(real_X, real_Y):
    with tf.GradientTape(persistent=True) as tape:
        # Generator Forward Passes
        fake_Y = G(real_X, training=True)
        fake_X = F(real_Y, training=True)

        cycled_X = F(fake_Y, training=True)
        cycled_Y = G(fake_X, training=True)

        # Discriminator Forward Passes
        disc_real_X = D_X(real_X, training=True)
        disc_fake_X = D_X(fake_X, training=True)

        disc_real_Y = D_Y(real_Y, training=True)
        disc_fake_Y = D_Y(fake_Y, training=True)

        # Losses
        g_loss = bce(tf.ones_like(disc_fake_Y), disc_fake_Y) + mae(real_Y, cycled_Y)
        f_loss = bce(tf.ones_like(disc_fake_X), disc_fake_X) + mae(real_X, cycled_X)

        dx_loss = bce(tf.ones_like(disc_real_X), disc_real_X) + bce(tf.zeros_like(disc_fake_X), disc_fake_X)
        dy_loss = bce(tf.ones_like(disc_real_Y), disc_real_Y) + bce(tf.zeros_like(disc_fake_Y), disc_fake_Y)

    # Apply Gradients
    g_grads = tape.gradient(g_loss, G.trainable_variables)
    f_grads = tape.gradient(f_loss, F.trainable_variables)
    dx_grads = tape.gradient(dx_loss, D_X.trainable_variables)
    dy_grads = tape.gradient(dy_loss, D_Y.trainable_variables)

    g_optimizer.apply_gradients(zip(g_grads, G.trainable_variables))
    f_optimizer.apply_gradients(zip(f_grads, F.trainable_variables))
    dx_optimizer.apply_gradients(zip(dx_grads, D_X.trainable_variables))
    dy_optimizer.apply_gradients(zip(dy_grads, D_Y.trainable_variables))

    return g_loss, f_loss, dx_loss, dy_loss

# ------------------------------
# Dummy Data for Testing
# ------------------------------

# Replace this with your actual dataset
sketches = np.random.rand(10, 256, 256, 3).astype(np.float32)
photos = np.random.rand(10, 256, 256, 3).astype(np.float32)

# ------------------------------
# Training Loop
# ------------------------------

epochs = 100
batch_size = 2
steps_per_epoch = len(sketches) // batch_size

for epoch in range(epochs):
    for step in range(steps_per_epoch):
        idx = np.random.randint(0, len(sketches), batch_size)
        real_X_batch = sketches[idx]
        real_Y_batch = photos[idx]
        g_loss, f_loss, dx_loss, dy_loss = train_step(real_X_batch, real_Y_batch)

    print(f"Epoch {epoch+1}/{epochs} | G_loss: {g_loss:.4f} | F_loss: {f_loss:.4f} | DX_loss: {dx_loss:.4f} | DY_loss: {dy_loss:.4f}")


Epoch 1/100 | G_loss: 1.1857 | F_loss: 1.1996 | DX_loss: 1.3291 | DY_loss: 1.3512
Epoch 2/100 | G_loss: 1.1588 | F_loss: 1.1772 | DX_loss: 1.2556 | DY_loss: 1.2864
Epoch 3/100 | G_loss: 1.1202 | F_loss: 1.1361 | DX_loss: 1.2067 | DY_loss: 1.2465
Epoch 4/100 | G_loss: 1.0613 | F_loss: 1.0647 | DX_loss: 1.1985 | DY_loss: 1.2296
Epoch 5/100 | G_loss: 0.9732 | F_loss: 0.9646 | DX_loss: 1.2337 | DY_loss: 1.2466
Epoch 6/100 | G_loss: 0.8604 | F_loss: 0.8617 | DX_loss: 1.2910 | DY_loss: 1.3024
Epoch 7/100 | G_loss: 0.7762 | F_loss: 0.7966 | DX_loss: 1.3390 | DY_loss: 1.3637
Epoch 8/100 | G_loss: 0.7697 | F_loss: 0.7918 | DX_loss: 1.3662 | DY_loss: 1.3868
Epoch 9/100 | G_loss: 0.8228 | F_loss: 0.8212 | DX_loss: 1.3873 | DY_loss: 1.3798
Epoch 10/100 | G_loss: 0.8844 | F_loss: 0.8496 | DX_loss: 1.4130 | DY_loss: 1.3764
Epoch 11/100 | G_loss: 0.9286 | F_loss: 0.8724 | DX_loss: 1.4270 | DY_loss: 1.3823
Epoch 12/100 | G_loss: 0.9538 | F_loss: 0.8981 | DX_loss: 1.4166 | DY_loss: 1.3901
Epoch 13/100 

In [12]:
# ------------------------------
# Save models after training
# ------------------------------
G.save("generator_G_SketchToPhoto.h5")
F.save("generator_F_PhotoToSketch.h5")
D_X.save("discriminator_DX_Sketch.h5")
D_Y.save("discriminator_DY_Photo.h5")

print("Models saved successfully.")


Models saved successfully.


In [28]:
import gradio as gr
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import os

# Load the trained generator (Sketch → Photo)
G_loaded = load_model("generator_G_SketchToPhoto.h5", compile=False)

# Sketches folder
SKETCHES_FOLDER = "/content/test/sketches"

# Get list of sketch filenames
sketch_filenames = sorted([
    f for f in os.listdir(SKETCHES_FOLDER)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

def preprocess(image):
    image = image.resize((256, 256))
    image = np.array(image) / 127.5 - 1  # Normalize to [-1, 1]
    return np.expand_dims(image, axis=0)

def postprocess(output):
    output = (output + 1) * 127.5  # Denormalize to [0, 255]
    output = np.clip(output[0], 0, 255).astype(np.uint8)
    return Image.fromarray(output)

def sketch_to_photo_dropdown(filename):
    sketch_path = os.path.join(SKETCHES_FOLDER, filename)
    sketch_image = Image.open(sketch_path).convert("RGB")
    input_tensor = preprocess(sketch_image)
    generated = G_loaded.predict(input_tensor)
    return postprocess(generated)

interface = gr.Interface(
    fn=sketch_to_photo_dropdown,
    inputs=gr.Dropdown(choices=sketch_filenames, label="Choose a Sketch Filename"),
    outputs=gr.Image(type="pil", label="Generated Photo"),
    title="Sketch to Photo using CycleGAN",
    description="This uses a pre-trained CycleGAN model to convert a sketch to a photo. Select a sketch to generate its corresponding photo."
)

interface.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2475333f8dc785b69f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
